In [1]:
import pandas as pd
import numpy as np
import requests
import time

In [ ]:
pip install pandas-gbq

## Connect to BQ


In [6]:
from google.oauth2 import service_account
from google.cloud import bigquery


In [7]:
credentials = service_account.Credentials.from_service_account_file(
'd:/Users/Ramona/Desktop/music-recommendation-system-24-3d0d21fb1f8b.json')
# music-recommendation-system-24-3d0d21fb1f8b.json is the service account JSON file. Save the file locally on your device and add the path here.
# make sure that the slashes in the path are '/' and not '\'

project_id = 'music-recommendation-system-24'
client = bigquery.Client(credentials= credentials,project=project_id)

##Query BQ

In [ ]:
query_job = client.query("""
   SELECT
      *
   FROM `music-recommendation-system-24.obama_playlists.barack_obamas_playlist_2019-2024_clean_enriched_csv`
   """)
#put the query in three quote marks

results = query_job.result() # Wait for the job to complete.

rows = [dict(row) for row in results]

# Convert the list of dictionaries to a DataFrame
df_bq = pd.DataFrame(rows)

##Getting the Access Token using Client_id and Client_secret

In [23]:
import requests
import base64

# Replace with your own client ID and client secret
CLIENT_ID = 'fffcb85f26d44129b1b51ec8edc4e0e0'
CLIENT_SECRET = '045459b5bf14446aa76c737eb617dd4f'

def get_access_token(client_id, client_secret):
    # Base64 encode client_id and client_secret
    auth_str = f"{client_id}:{client_secret}"
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()

    # Define the token endpoint and headers
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {b64_auth_str}"
    }
    data = {
        "grant_type": "client_credentials"
    }

    # Make the request to get the token
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        token = response.json().get('access_token')
        return token
    else:
        raise Exception("Could not get access token")

In [24]:
# Call the function to get the access token

token = get_access_token(CLIENT_ID, CLIENT_SECRET)

# Attempt to get all artists' info and store them as lists instead of a value

In [ ]:
# Create a dictionary to store data. Later on will be converted to a Data Frame

d = {
    'track_id' : [],
    'artist_id' : [],
    'artist_name' : [],
    'artist_followers' : [],
    'artist_popularity' : []
}

# Define the batch size
batch_size = 25

# Iterate over 'track_id' values in batches
for start in range(0, len(df_bq['track_id']), batch_size):
    track_ids_batch = df_bq['track_id'][start:start + batch_size]

    # Iterate over each track_id in the current batch
    for id in track_ids_batch:

        # Add track_id to dictionary
        d['track_id'].append(id)

        # use the track id to make an API call to get the artist id
        endpoint = f"https://api.spotify.com/v1/tracks/{id}"

        while True:
            response = requests.get(endpoint, headers={'Authorization': f"Bearer {token}"})

            # If we hit the rate limit, wait for the specified time and retry
            if response.status_code != 200:
                retry_after = int(response.headers.get('Retry-After', 30))  # Default to 30 seconds if not provided
                print(f"Rate limit hit. Retrying after {retry_after} seconds...")
                time.sleep(retry_after)
                continue  # Retry the request

            elif response.status_code == 200:
                track_response = response.json()
                break  # Exit the loop if the request is successful
            else:
                print(f"Error: {response.status_code} - {response.text}")
                break  # Exit loop on any other error

    # creating lists to store in the info of multiple artists

    art_names = []
    art_followers = []
    art_ids = []
    art_popularities = []

    # get a list of all artists' ids

    for artist in track_response['artists'] :

      art_ids.append(artist['id'])

    # You should iterate over all elements of the list that contains all artists_ids and make an API call for each of them
    # artist endpoint to get artist's info

    for artist_id in art_ids:
        artist_endpoint = f"https://api.spotify.com/v1/artists/{artist_id}"

        while True:
            artist_response = requests.get(artist_endpoint, headers={'Authorization': f"Bearer {token}"})

            # Check for rate limit and wait if needed
            if artist_response.status_code != 200:
                retry_after = int(artist_response.headers.get('Retry-After', 30))
                print(f"Rate limit hit. Retrying after {retry_after} seconds...")
                time.sleep(retry_after)
                continue  # Retry the request

            elif artist_response.status_code == 200:
                artist_data = artist_response.json()
                break  # Exit the loop if the request is successful
            else:
                print(f"Error: {artist_response.status_code} - {artist_response.text}")
                break  # Exit loop on any other error

        # Extract and store artist data
        art_names.append(artist_data['name'])
        art_followers.append(artist_data['followers']['total'])
        art_popularities.append(artist_data['popularity'])

    # Append data to the dictionary
    d['artist_name'].append(art_names)
    d['artist_followers'].append(art_followers)
    d['artist_popularity'].append(art_popularities)
    d['artist_id'].append(art_ids)



In [ ]:
pd.DataFrame(d)

In [ ]:
df_artists = pd.DataFrame(d)
df_artists.info()

In [18]:
#merge df_obama_21 with df_artists
df_bq = pd.merge(df_bq, df_artists, on='track_id')

In [ ]:
#save merged to gbq
# Define your project ID and BigQuery table ID (including dataset name)
project_id = 'music-recommendation-system-24'
table_id = 'obama_playlists.barack_obamas_playlist_2019-2024_artist_enriched_v1'  # Replace 'your_dataset_name' with your actual dataset name

# Load the credentials from your service account file
credentials = service_account.Credentials.from_service_account_file(
    'd:/Users/Ramona/Desktop/music-recommendation-system-24-3d0d21fb1f8b.json'
)

# Save DataFrame to BigQuery
df_bq.to_gbq(
    destination_table=table_id,
    project_id=project_id,
    credentials=credentials,
    if_exists='replace'  # Use 'append' to add data if the table already exists
)

print("DataFrame saved to BigQuery table 'barack_obamas_playlist_2019-2024_artist_enriched_v1'.")